In [ ]:
import torch
import torch.nn.functional as F

def fgsm_attack(data, epsilon, gradient):
    return data + epsilon * gradient.sign()

def pgd_attack(model, data, label, epsilon=0.1, alpha=0.01, iters=10):
    data_adv = data.clone().detach().requires_grad_(True)
    for _ in range(iters):
        output = model(data_adv)
        loss = F.cross_entropy(output, label)
        model.zero_grad()
        loss.backward()
        data_adv = data_adv + alpha * data_adv.grad.sign()
        perturbation = torch.clamp(data_adv - data, -epsilon, epsilon)
        data_adv = torch.clamp(data + perturbation, 0, 1).detach().requires_grad_(True)
    return data_adv


def evaluate(model, X, y):
    model.eval()
    correct = 0
    with torch.no_grad():
        for i in range(len(X)):
            output = model(X[i].unsqueeze(0))
            pred = torch.argmax(output, dim=1)
            correct += (pred.item() == y[i].item())
    return correct / len(y)